In [0]:
dbutils.fs.mount(source='wasbs://landing@week11assignment.blob.core.windows.net',
mount_point='/mnt/week11assignment/landing',
extra_configs={'fs.azure.account.key.week11assignment.blob.core.windows.net':'Xw4FQ3bo780rfAQr/wCs0ynLsL0lq+H2s8NWo8iVv4JSPoXePJD9f3eRnFU9A3Gi5bv143kX5/h++AStOtFPxQ=='})

In [0]:
dbutils.fs.ls('/mnt/week11assignment/landing')

In [0]:
dbutils.fs.mount(source='wasbs://staging@week11assignment.blob.core.windows.net',
mount_point='/mnt/week11assignment/staging',
extra_configs={'fs.azure.account.key.week11assignment.blob.core.windows.net':'Xw4FQ3bo780rfAQr/wCs0ynLsL0lq+H2s8NWo8iVv4JSPoXePJD9f3eRnFU9A3Gi5bv143kX5/h++AStOtFPxQ=='})

In [0]:
dbutils.fs.mount(source='wasbs://reporting@week11assignment.blob.core.windows.net',
mount_point='/mnt/week11assignment/reporting',
extra_configs={'fs.azure.account.key.week11assignment.blob.core.windows.net':'Xw4FQ3bo780rfAQr/wCs0ynLsL0lq+H2s8NWo8iVv4JSPoXePJD9f3eRnFU9A3Gi5bv143kX5/h++AStOtFPxQ=='})

In [0]:
orders_schema = 'order_id string, order_date date, order_customer_id string, order_status string'

orders_df = spark.read.format("csv") \
.schema(orders_schema) \
.load("/mnt/week11assignment/landing/orders-230715-163711.csv")

In [0]:
display(orders_df)

In [0]:
customers_schema = 'customer_id string, customer_fname string, customer_lname string, customer_email string, customer_password string, customer_street string, customer_city string, customer_state string,customer_zipcode string'

customers_df = spark.read.format("csv") \
.schema(customers_schema) \
.load("/mnt/week11assignment/landing/customers-230715-163711.csv")

In [0]:
display(customers_df)

In [0]:
from pyspark.sql.functions import year, month

orders_df = orders_df.withColumn("order_year",year("order_date")).withColumn("order_month", month("order_date"))

In [0]:
display(orders_df)

In [0]:
orders_df.write\
.format("parquet")\
.mode("overwrite")\
.partitionBy("order_year","order_status")\
.save("/mnt/week11assignment/staging/order")

In [0]:
customers_df.write\
.format("parquet")\
.mode("overwrite")\
.partitionBy("customer_state")\
.save("/mnt/week11assignment/staging/customer")

In [0]:
customers_df_stg = spark.read.format("parquet") \
.option("header","true")\
.load("/mnt/week11assignment/staging/customer")

In [0]:
display(customers_df_stg)

In [0]:
orders_df_stg = spark.read.format("parquet") \
.option("header","true")\
.load("/mnt/week11assignment/staging/order")

In [0]:
display(orders_df_stg)

In [0]:
from pyspark.sql.functions import broadcast, expr

joined_df = orders_df_stg.join(broadcast(customers_df_stg), expr("order_customer_id=customer_id"))

reporting_df = joined_df.select("order_id","order_date","order_customer_id","order_month","order_year","order_status","customer_fname","customer_lname","customer_city","customer_zipcode","customer_state")

In [0]:
display(reporting_df)

In [0]:
reporting_df.write\
.format("parquet")\
.mode("overwrite")\
.partitionBy("order_year","customer_state","customer_city","order_status")\
.save("/mnt/week11assignment/reporting")

In [0]:
order_cust_rpt = spark.read.format("parquet") \
.option("header","true")\
.load("/mnt/week11assignment/reporting")

display(order_cust_rpt)

In [0]:
order_cust_rpt.select(expr("order_customer_id").alias("customer_id"),expr("concat(customer_fname,'',customer_lname)").alias("customer_name"),"order_date","customer_city","customer_state","order_status","order_year","order_month") \
.createOrReplaceTempView("order_cust_vw")

In [0]:
spark.sql("select * from order_cust_vw limit 10").show()

In [0]:
spark.sql("cache table order_cust_vw")

In [0]:
%sql
--Ans 6b - running the required query
select distinct customer_name from order_cust_vw where customer_state='TX' AND order_status='COMPLETE' AND order_year='2014'

In [0]:
dbutils.widgets.dropdown(name = 'state_input', defaultValue='TX', choices=['TX', 'NY', 'CA'], label= "State:")

dbutils.widgets.dropdown(name = 'status_input', defaultValue='COMPLETE', choices=['COMPLETE', 'PENDING_PAYMENT', 'CLOSED', 'PROCESSING'],label= "Status:")

dbutils.widgets.text(name = 'year_input', defaultValue= "2014", label= "Year:")

In [0]:
state = dbutils.widgets.get('state_input')
status = dbutils.widgets.get('status_input')
year = dbutils.widgets.get('year_input')

In [0]:
display(spark.sql(f"SELECT distinct customer_name FROM order_cust_vw WHERE customer_state = '{state}' AND order_status = '{status}' AND order_year = '{year}'"))

In [0]:
dbutils.widgets.dropdown("month_input", "1", ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"], "Month:")

month = dbutils.widgets.get("month_input")

In [0]:
display(spark.sql(f"SELECT count(customer_id) FROM order_cust_vw WHERE customer_state = '{state}' AND order_status = '{status}' AND order_year = '{year}' AND order_month = '{month}'"))